<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/code/textclassification/cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Embedding,LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.datasets import imdb
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 1.参数设置

In [2]:
#embedding 参数
maxlen=100
embedding_size=200

#卷积参数
kernel_size=5
filters=128
pool_size=4

#LSTM参数
lstm_output_size=100

#训练参数
batch_size=128
epochs=2

# 2.数据预处理及训练数据准备

In [3]:
def textToChars(filePath):
  """
  读取文本文件并进行处理
  :param filePath:文件路径
  :return:
  """
  lines = []
  df=pd.read_excel(filePath,header=None)
  df.columns=['content']
  for index, row in df.iterrows():
    row=row['content']
    row = re.sub("[^\u4e00-\u9fa5]", "", str(row))  # 只保留中文
    lines.append(list(str(row)))
  return lines


def getWordIndex(vocabPath):
  """
  获取word2Index,index2Word
  :param vocabPath:词汇文件
  :return:
  """
  word2Index = {}
  with open(vocabPath, encoding="utf-8") as f:
    for line in f.readlines():
      word2Index[line.strip()] = len(word2Index)
  index2Word = dict(zip(word2Index.values(), word2Index.keys()))
  return word2Index, index2Word


def lodaData(posFile, negFile, word2Index):
  """
  获取训练数据
  :param posFile:正样本文件
  :param negFile:负样本文件
  :param word2Index:
  :return:
  """
  posLines = textToChars(posFile)
  negLines = textToChars(negFile)
  textLines=posLines+negLines
  print("正样本数量%d,负样本数量%d"%(len(posLines),len(negLines)))
  posIndexLines = [[word2Index[word] if word2Index.get(word) else 0 for word in line] for line in posLines]
  negIndexLines = [[word2Index[word] if word2Index.get(word) else 0 for word in line] for line in negLines]
  lines = posIndexLines + negIndexLines
  print("训练样本和测试样本共：%d 个"%(len(lines)))
  # lens = [len(line) for line in lines]
  labels = [1] * len(posIndexLines) + [0] * len(negIndexLines)
  padSequences = sequence.pad_sequences(lines, maxlen=maxlen, padding="post", truncating="post")
  X_train,X_test,y_train,y_test=train_test_split(padSequences,np.array(labels),test_size=0.2,random_state=42)
  return (textLines,labels),(X_train,X_test,y_train,y_test)

In [4]:
vocabPath="/content/drive/My Drive/data/vocab.txt"
negFilePath="/content/drive/My Drive/data/text_classify/sentiment/neg.xls"
posFilePath="/content/drive/My Drive/data/text_classify/sentiment/pos.xls"
word2Index, index2Word=getWordIndex(vocabPath)
(textLines,labels),(X_train,X_test,y_train,y_test)=lodaData(posFile=posFilePath,negFile=negFilePath,word2Index=word2Index)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

正样本数量10677,负样本数量10428
训练样本和测试样本共：21105 个
(16884, 100) (4221, 100) (16884,) (4221,)


In [5]:
X_train[0],y_train[0]

(array([6821, 3221,  671, 3315, 2769,  702,  782, 6371,  711, 2523, 2141,
        4500, 4638,  741,  852, 1079, 2159, 3300,  763, 5042, 1296,  679,
        6814, 2828,  809, 1184, 1762, 2110, 3413, 7027, 2110, 6814, 4638,
        5632, 3131, 4638, 4761, 6399, 7028, 3946,  749, 7390, 4708, 2399,
        7977, 1469, 4852,  833, 4384, 1862, 2128, 6871, 5632, 3131, 4638,
        2692, 6399, 6632, 3341, 6632, 2483,  749, 4684, 1168, 1724, 2335,
        1765, 7448,  782, 5102, 1762, 5632, 4197,  704, 3221, 1963, 3634,
        4638, 3953, 2207, 5445, 5546, 2483, 2769,  812, 3187, 1213, 2850,
        2834,  852, 3221, 1963, 3362, 2769,  812, 2958, 2995,  671,  763,
        5632], dtype=int32), 1)

# 3.网络结构搭建及模型训练

In [6]:
model=Sequential()
model.add(Embedding(len(word2Index),embedding_size,input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters,kernel_size,padding="valid",activation="relu",strides=1))
model.add(MaxPooling1D(pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print("开始训练")
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test,y_test))

开始训练
Epoch 1/2
132/132 [==============================] - 8s 57ms/step - loss: 0.5012 - accuracy: 0.7406 - val_loss: 0.3162 - val_accuracy: 0.8778
Epoch 2/2
132/132 [==============================] - 7s 53ms/step - loss: 0.2644 - accuracy: 0.9006 - val_loss: 0.2633 - val_accuracy: 0.8965


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 200)          4225600   
_________________________________________________________________
dropout (Dropout)            (None, 100, 200)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 128)           128128    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               91600     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
_________________________________________________________________
activation (Activation)      (None, 1)                 0

In [14]:
def predict_one(sentence,model,word2Index):
  sentence=re.sub("[^\u4e00-\u9fa5]", "", str(sentence))  # 只保留中文
  # print(sentence)
  sentence=[word2Index[word] if word2Index.get(word) else 0 for word in sentence]
  sentence=sentence+[0]*(maxlen-len(sentence)) if len(sentence)<maxlen else sentence[0:300]
  # print(sentence)
  sentence=np.reshape(np.array(sentence),(-1,len(sentence))) 
  pred_prob=model.predict(sentence)
  label = 1 if pred_prob[0][0]>0.5 else 0
  print(label)
  return label


In [15]:
sentence="一次很不爽的购物，页面上说是第二天能到货，结果货是从陕西发出的，卖家完全知道第二天根本到不了货。多处提到送货入户还有100%送货入户也没有兑现，与客服联系多日，还是把皮球踢到快递公司。算是一个教训吧。"
predict_one(sentence,model,word2Index)

0
